### Test methods that work with accounts

In [1]:
from dexter.DB import DB, Account, Entry, Transaction, Document, Tag, Category

import re

In [2]:
DB.init()

In [3]:
DB.open('dev')

### Match Account Names

In [4]:
Account.objects(name__contains='checking')

[<Account: checking assets>]

In [5]:
Account.objects(name__contains='food')

[<Account: expenses:food expenses>, <Account: groceries expenses>, <Account: restaurant expenses>]

In [6]:
for a in DB.find_account('food'):
    print(a)

expenses:food expenses
groceries expenses
restaurant expenses


In [7]:
DB.account_name_parts()

{'assets',
 'bank',
 'car',
 'chase',
 'checking',
 'entertainment',
 'equity',
 'expenses',
 'food',
 'fuel',
 'groceries',
 'home',
 'household',
 'income',
 'interest',
 'liabilities',
 'mortgage',
 'payment',
 'restaurant',
 'savings',
 'utility',
 'visa',
 'yard',
 'yoyodyne'}

In [8]:
DB.account_name_parts('expenses')

set()

In [9]:
dct = DB.account_names()

In [10]:
dct

{'equity': {'equity'},
 'assets:bank:checking': {'assets:bank:checking'},
 'checking': {'assets:bank:checking'},
 'assets': {'assets:bank:checking', 'assets:bank:savings'},
 'bank': {'assets:bank:checking', 'assets:bank:savings'},
 'assets:bank:savings': {'assets:bank:savings'},
 'savings': {'assets:bank:savings'},
 'expenses:car': {'expenses:car'},
 'expenses': {'expenses:car',
  'expenses:car:fuel',
  'expenses:car:payment',
  'expenses:entertainment',
  'expenses:food',
  'expenses:food:groceries',
  'expenses:food:restaurant',
  'expenses:home',
  'expenses:home:household',
  'expenses:home:mortgage',
  'expenses:home:utility',
  'expenses:home:yard'},
 'car': {'expenses:car', 'expenses:car:fuel', 'expenses:car:payment'},
 'expenses:car:payment': {'expenses:car:payment'},
 'payment': {'expenses:car:payment'},
 'expenses:car:fuel': {'expenses:car:fuel'},
 'fuel': {'expenses:car:fuel'},
 'expenses:entertainment': {'expenses:entertainment'},
 'entertainment': {'expenses:entertainment'

In [11]:
dct['expenses']

{'expenses:car',
 'expenses:car:fuel',
 'expenses:car:payment',
 'expenses:entertainment',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'expenses:home:household',
 'expenses:home:mortgage',
 'expenses:home:utility',
 'expenses:home:yard'}

In [12]:
len(dct['expenses'])

12

In [13]:
dct = DB.account_names('expenses')

In [14]:
dct

{}

In [15]:
len(dct)

0

In [16]:
# DB.account_groups()

In [17]:
# DB.account_groups(['expenses'])

In [18]:
# DB.account_groups(['expenses:1'])

In [19]:
# DB.account_groups(['expenses:2'])

In [20]:
# DB.account_groups(['expenses:3'])

In [21]:
# DB.account_groups(['expenses:food'])

In [22]:
# DB.account_groups(['expenses:food:2'])

In [23]:
# DB.account_groups(['expenses:food:3'])

In [24]:
# DB.account_groups(['expenses','income'])

In [25]:
# DB.account_groups(['expenses:1','income:1'])

In [26]:
# DB.account_groups(['expenses:2','income:2'])

In [27]:
Account.objects(name__exact='expenses:food:groceries')

[<Account: groceries expenses>]

In [28]:
Account.objects(abbrev__exact='groceries')

[<Account: groceries expenses>]

In [29]:
s = 'expenses:food:groceries'

In [30]:
if acct := Account.objects(name__exact=s) or Account.objects(abbrev__exact=s):
    print(acct[0].name)

expenses:food:groceries


In [31]:
DB.fullname('expenses:food:groceries')

'expenses:food:groceries'

In [32]:
DB.fullname('groceries')

'expenses:food:groceries'

In [33]:
DB.fullname('groc')

In [34]:
Account.objects.get(name='assets:bank:checking')

<Account: checking assets>

In [35]:
Account.objects.get(abbrev='checking')

<Account: checking assets>

Bummer -- would be nice if this returned None instead of raising an exception:

In [36]:
# Account.objects.get(abbrev='foo')

In [37]:
Account.DoesNotExist

dexter.DB.DoesNotExist

In [38]:
Account.objects.first()

<Account: equity equity>

In [39]:
def abbrev(a):
    try:
        acct = Account.objects.get(name=a)
        res = acct.abbrev
    except Account.DoesNotExist:
        res = a
    return res

In [40]:
abbrev('assets:bank:checking')

'checking'

In [41]:
abbrev('assets:bank:whatever')

'assets:bank:whatever'

In [42]:
abbrev('expenses:personal:bob')

'expenses:personal:bob'

### Balances

In [43]:
DB.select(Entry, account='expenses:food*')

[<Entry: <En 2024-04-26 expenses:food:groceries +$15.0 []>>, <Entry: <En 2024-04-24 expenses:food:groceries +$15.0 []>>, <Entry: <En 2024-04-22 expenses:food:restaurant +$50.0 []>>, <Entry: <En 2024-04-23 expenses:food:groceries +$15.0 []>>, <Entry: <En 2024-04-01 expenses:food:restaurant +$70.0 []>>, <Entry: <En 2024-04-03 expenses:food:restaurant -$35.0 []>>, <Entry: <En 2024-04-12 expenses:food:groceries +$65.0 []>>]

In [44]:
for e in DB.select(Entry, account='expenses:food*'):
    print(e)

<En 2024-04-26 expenses:food:groceries +$15.0 []>
<En 2024-04-24 expenses:food:groceries +$15.0 []>
<En 2024-04-22 expenses:food:restaurant +$50.0 []>
<En 2024-04-23 expenses:food:groceries +$15.0 []>
<En 2024-04-01 expenses:food:restaurant +$70.0 []>
<En 2024-04-03 expenses:food:restaurant -$35.0 []>
<En 2024-04-12 expenses:food:groceries +$65.0 []>


In [45]:
for t in DB.select(Transaction, tag='#budget'):
    print(t)

In [46]:
DB.balance('food')

0

In [49]:
DB.balance('food', nobudget=True)

ERROR:root:--nobudget not implemented in DB.balance


0

In [50]:
DB.balance('food', ending='2024-01-31')

0

In [51]:
DB.balance('food', ending='2024-01-31', nobudget=True)

ERROR:root:--nobudget not implemented in DB.balance


0

In [52]:
for e in DB.select(Entry, account='expenses:car'):
    print(e)

<En 2024-04-05 expenses:car:payment +$400.0 []>
<En 2024-04-02 expenses:car +$5.0 []>
<En 2024-04-03 expenses:car:fuel +$50.0 []>


In [53]:
DB.select(Entry, account='expenses:car').sum('amount')

455.0

In [ ]:
# Account(name='expenses:car:fuel:gas', category=Category.E).save()

In [ ]:
# Account(name='expenses:car:fuel:electric', category=Category.E).save()

In [54]:
for a in Account.objects(category = Category.E):
    print(a.name)

expenses:car
expenses:car:payment
expenses:car:fuel
expenses:entertainment
expenses:food
expenses:food:groceries
expenses:food:restaurant
expenses:home
expenses:home:mortgage
expenses:home:household
expenses:home:utility
expenses:home:yard


In [55]:
    def expand_node(node, level):
        '''
        A command line argument had an account name and level.  Return the
        list of accounts below to the specified level.
        '''
        res = []
        for acct in Account.objects(name__startswith=node):
            name = acct.name
            tail = name[len(node):]
            print(name, tail, tail.count(':'))
            if tail.count(':') <= level:
                res.append(name)
        return res

In [56]:
expand_node('expenses:car', 1)

expenses:car  0
expenses:car:payment :payment 1
expenses:car:fuel :fuel 1


['expenses:car', 'expenses:car:payment', 'expenses:car:fuel']

In [57]:
DB.expand_node('expenses:car:1')

['expenses:car', 'expenses:car:payment', 'expenses:car:fuel']

In [58]:
DB.expand_node('expenses:car:2')

['expenses:car', 'expenses:car:payment', 'expenses:car:fuel']

In [59]:
DB.expand_node('expenses:car:0')

['expenses:car']

In [60]:
re.match(r'.*:\d+', 'expenses:car:1')

<re.Match object; span=(0, 14), match='expenses:car:1'>

In [61]:
re.match(r'(.*):(\d+)','expenses:car:10').groups()

('expenses:car', '10')

In [62]:
DB.account_glob('expenses:car')

['expenses:car']

In [63]:
DB.account_glob('groceries')

['expenses:food:groceries']

In [64]:
DB.account_glob('expenses:car:')

['expenses:car:']

In [65]:
DB.account_glob('expenses:cart')

In [66]:
DB.account_glob('expenses:car:1')

['expenses:car', 'expenses:car:payment', 'expenses:car:fuel']

In [67]:
DB.account_glob('expenses:car:2')

['expenses:car', 'expenses:car:payment', 'expenses:car:fuel']

In [68]:
DB.account_glob('expenses:car:0')

['expenses:car']

In [69]:
DB.account_glob('@car')

['@car']

In [70]:
DB.account_glob('expenses:car:')

['expenses:car:']

In [71]:
DB.account_glob('expenses:car:0')

['expenses:car']

In [72]:
def account_args(s):
    if s.startswith('@'):
        res = { 'account__regex': f'\\b{s[1:]}\\b' }
    elif s.endswith(':'):
        res = { 'account__regex': f'^{s[:-1]}.*$' }
    else:
        res = { 'account__regex': f'^{s}$'}
    return res

In [73]:
account_args('expenses:car')

{'account__regex': '^expenses:car$'}

In [74]:
account_args('expenses:car:')

{'account__regex': '^expenses:car.*$'}

In [75]:
account_args('@car')

{'account__regex': '\\bcar\\b'}

In [78]:
lst = Transaction.objects(pamount__lte=50)

In [79]:
lst

[<Transaction: <Tr 2024-04-26 liabilities:chase:visa -> expenses:food:groceries $15.0 Long's Meat Market []>>, <Transaction: <Tr 2024-04-24 liabilities:chase:visa -> expenses:food:groceries $15.0 Newman's []>>, <Transaction: <Tr 2024-04-22 liabilities:chase:visa -> expenses:food:restaurant $50.0 Hey Neighbor []>>, <Transaction: <Tr 2024-04-04 liabilities:chase:visa -> expenses:entertainment $20.0 Netflix []>>, <Transaction: <Tr 2024-04-30 income:interest -> assets:bank:checking $0.68 Interest []>>, <Transaction: <Tr 2024-04-23 assets:bank:checking -> expenses:food:groceries $15.0 Market []>>, <Transaction: <Tr 2024-04-30 income:interest -> assets:bank:savings $1.59 Interest []>>, <Transaction: <Tr 2024-04-02 liabilities:chase:visa -> expenses:car $5.0 Wash-it Express []>>, <Transaction: <Tr 2024-04-03 expenses:food:restaurant -> assets:bank:checking $35.0 Transfer From Venmo []>>, <Transaction: <Tr 2024-04-03 assets:bank:checking -> expenses:car:fuel $50.0 Kayenta Chevron []>>, <Transa

In [80]:
len(lst)

14

In [81]:
type(lst)

mongoengine.queryset.queryset.QuerySet

In [84]:
lst(pdate__lte='2024-04-03')

[<Transaction: <Tr 2024-04-02 liabilities:chase:visa -> expenses:car $5.0 Wash-it Express []>>, <Transaction: <Tr 2024-04-03 expenses:food:restaurant -> assets:bank:checking $35.0 Transfer From Venmo []>>, <Transaction: <Tr 2024-04-03 assets:bank:checking -> expenses:car:fuel $50.0 Kayenta Chevron []>>]

In [85]:
for t in lst(pdate__lte='2024-04-03'):
    print(t)

<Tr 2024-04-02 liabilities:chase:visa -> expenses:car $5.0 Wash-it Express []>
<Tr 2024-04-03 expenses:food:restaurant -> assets:bank:checking $35.0 Transfer From Venmo []>
<Tr 2024-04-03 assets:bank:checking -> expenses:car:fuel $50.0 Kayenta Chevron []>


In [94]:
dct = {'a': 0, 'b': 1}

In [98]:
# del dct['a']     <- works if 'a' in dict, raises exception if not
# dct.pop('a)      <- same
dct.pop('a', None)    # what we want -- returns None if 'a' not in dict

In [99]:
dct

{'b': 1}

In [113]:
lst1 = Transaction.objects(pamount__lte=50, pdate__lte='2024-04-03')

In [102]:
for t in lst1:
    print(t)

<Tr 2024-04-02 liabilities:chase:visa -> expenses:car $5.0 Wash-it Express []>
<Tr 2024-04-03 expenses:food:restaurant -> assets:bank:checking $35.0 Transfer From Venmo []>
<Tr 2024-04-03 assets:bank:checking -> expenses:car:fuel $50.0 Kayenta Chevron []>


In [114]:
lst2 = Transaction.objects(pamount__lte=50, pdate__gte='2024-04-25')

In [106]:
for t in lst2:
    print(t)

<Tr 2024-04-26 liabilities:chase:visa -> expenses:food:groceries $15.0 Long's Meat Market []>
<Tr 2024-04-30 income:interest -> assets:bank:checking $0.68 Interest []>
<Tr 2024-04-30 income:interest -> assets:bank:savings $1.59 Interest []>


In [117]:
# for t in lst1+lst2:     <- QuerySet is not list-like :-(
#     print(t)